# Database <----> Pandas

# Announcements - Wednesday, November 30
* Download the files on Canvas
* **Course evaluations** coming very soon. Check your email.
* Final Exam
  * December 19
  * Please fill out the [exam conflict form](https://cs220.cs.wisc.edu/f22/surveys.html)
    * Conflict form will be taken down on **Dec 14**
    * McBurney accomodation conflicts need to make appointment with T&E by **Dec 9**.
    * We expect you to resolve conflicts before then
    * There will be a new page for illness/emergencies
  * If you have completed Lab P12 **and P12** before your lab section meets 
    * You will be granted credit for lab
    * Special offer this week only due to Thanksgiving break
    * Email your [lab instructor](https://canvas.wisc.edu/courses/322078/pages/lab-to-ta-slash-peer-mentor-mapping?module_item_id=5407173) to let them know they should look on Gradescope to verify you completed the project
* [Zoom Link](https://uwmadison.zoom.us/j/9741859842?pwd=OURuZnZuL0lhYlJkNVJHR1pLeUQwUT09)
  * Projector Only
  * No Audio
  * The class is not livestreamed 


In [1]:
# Run this cell to make the emphasized text red and use the full width of the screen
from IPython.core.display import HTML
HTML('<style>em { color: red; }</style> <style>.container {width:100% !important; }</style>')

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect("worksheet.db")

In [4]:
# this function provides a shortcut to making queries
# assumes we have access to a connection object - conn

def qry(QUERY):
    '''QUERY is a string containing SQL, conn is a global connection variable'''
    return pd.read_sql(QUERY, conn)

In [5]:
pd.read_sql("SELECT * from sqlite_master", conn)

,type,name,tbl_name,rootpage,sql
0,table,hydrants,hydrants,2,"CREATE TABLE ""hydrants"" (\n""year"" INTEGER,\n ..."
1,table,trees,trees,3,"CREATE TABLE ""trees"" (\n""tree"" TEXT,\n ""x"" IN..."
2,table,species,species,4,"CREATE TABLE ""species"" (\n""code"" TEXT,\n ""spe..."


In [6]:
# Remember that one database can hold several tables
hydrants = qry("SELECT * FROM hydrants")
trees = qry("SELECT * FROM trees")
species = qry("SELECT * FROM species")

In [7]:
# this is made-up data, but is inspired by an actual City of Madison database!
trees

,tree,x,y,species,diameter,priority
0,A,10,4,m,8,71
1,B,20,4,m,10,100
2,C,30,4,p,6,30
3,D,40,4,p,8,40
4,E,50,4,m,12,99


https://data-cityofmadison.opendata.arcgis.com/datasets/b700541a20e446839b18d62426c266a3/explore?location=43.072110%2C-89.405159%2C18.00

In [8]:
# Databases typically split up data into manageable pieces
# It may be more efficient to keep the species codes separate, since they are rarely updated
species

,code,species
0,m,maple
1,p,pine


In [9]:
# The City of Madison keeps data on fire hydrants!
hydrants

,year,color,style,owner,alt,active
0,1999,red,K-81,private,1179,0
1,2000,red,M-3,public,1065,0
2,2001,green,Pacer,private,1058,1
3,2010,blue,Pacer,public,1081,1
4,2014,blue,Pacer,public,1052,1
5,2018,blue,Pacer,public,1109,1


https://data-cityofmadison.opendata.arcgis.com/datasets/54c4877f16084409849ebd5385e2ee27_6/explore?location=43.071084%2C-89.403280%2C17.00

### 1a. *Without* running this cell - *predict* the output of the following statement

In [12]:
trees[trees["priority"] > 90][["x", "y"]]   


,x,y
1,20,4
4,50,4


### 1b. *Convert* the statement to an equivalent *SQL* querry.

In [14]:
trees
# your answer here
qry("""
SELECT x,y
FROM trees
WHERE priority > 90

""")

,x,y
0,20,4
1,50,4


----
### 2a. *Predict* the output of the following *SQL* querry

In [15]:
qry("SELECT x+y FROM trees WHERE species = 'm'")

,x+y
0,14
1,24
2,54


### 2b. *Convert* the querry into an equivalent *pandas* statement.

In [23]:
trees
# boolean
trees.species =='m'
# x
trees[trees.species == 'm'].x
# add
xy=trees[trees.species == 'm'].x + trees[trees.species == 'm'].y
# build Dataframe
pd.DataFrame(xy,columns=['x+y'])

,x+y
0,14
1,24
4,54


----
### 3a. *Predict* the output of the following *pandas* statements

In [24]:
cd = species["code"][species["species"]=="maple"].iloc[0]
trees[trees["species"] == cd]["tree"]

0    A
1    B
4    E
Name: tree, dtype: object

### 3b. *Convert* the statements into an equivalent *SQL* querry.

In [37]:
cd = qry("""
SELECT *
FROM species
WHERE species = 'maple'

""")['code'].iloc[0]

qry("""
SELECT tree
FROM trees
WHERE species == 'm'
""")
# using 'm' is hardcoing
qry("""
SELECT *
FROM trees
WHERE species == '{}'
""".format(cd))

,tree,x,y,species,diameter,priority
0,A,10,4,m,8,71
1,B,20,4,m,10,100
2,E,50,4,m,12,99


In [34]:
# how to pass variable into string
'my {} hurts'.format('ass')

'my ass hurts'

----
### 4a. *Predict* the output of the following querry

In [38]:
qry("SELECT species FROM trees ORDER BY priority DESC")

,species
0,m
1,m
2,m
3,p
4,p


### 4.b *Convert* the querry code to *Pandas*

In [43]:
# original
trees
# sort
trees.sort_values('priority',ascending = False)
# species column
trees.sort_values('priority',ascending = False).species

1    m
4    m
0    m
3    p
2    p
Name: species, dtype: object

----
### 5a. *Predict* the output of the following code

In [44]:
list(qry("SELECT tree, priority FROM trees ORDER BY priority DESC LIMIT 1").iloc[0])

['B', 100]

### 5.b *Convert* the above code to *Pandas*

In [54]:
# original
trees
# sort
trees.sort_values('priority', ascending = False)
# first row
trees.sort_values('priority', ascending = False).iloc[0]
# columns
trees.sort_values('priority', ascending = False).iloc[0].loc[['tree','priority']]
# list
list(trees.sort_values('priority', ascending = False).iloc[0].loc[['tree','priority']])


['B', 100]

----
### 6a. *Predict* the output of the following code

In [84]:
# qry("""SELECT COUNT(SPECIES) AS c1,
# COUNT(DISTINCT SPECIES) as c2
# FROM trees""")

### 6b. *Convert* the above code to *Pandas*

----
### 7a. *Predict* the output of the following code

In [55]:
3qry("""SELECT species, COUNT(SPECIES) AS count,
AVG(diameter) AS size
FROM trees
GROUP BY species ORDER BY count DESC""")

,species,count,size
0,m,3,10.0
1,p,2,7.0


### 7b. *Convert* the above code to *Pandas*

In [56]:
# part 1: species list
species_list = list(pd.unique(trees['species']))

species_list

['m', 'p']

In [60]:
# part 2: diameter - size
#         new command - groupby

# GROUP BY and aggragate

trees.groupby("species").mean()
# diamater
trees.groupby("species").mean().diameter
# list
size_list=list(trees.groupby("species").mean().diameter)

In [ ]:
# part 3: count


In [62]:
# part 4: make a DataFrame from a dict of lists
pd.DataFrame({'Species':species_list,'Size':size_list})

,Species,Size
0,m,10.0
1,p,7.0


----
# Additional Exercises: 
### *Predict* the output of the following statements

In [163]:

hydrants

,year,color,style,owner,alt,active
0,1999,red,K-81,private,1179,0
1,2000,red,M-3,public,1065,0
2,2001,green,Pacer,private,1058,1
3,2010,blue,Pacer,public,1081,1
4,2014,blue,Pacer,public,1052,1
5,2018,blue,Pacer,public,1109,1


In [165]:
#qry("SELECT color, year FROM hydrants WHERE color = 'blue' ")

,color,year
0,blue,2010
1,blue,2014
2,blue,2018


In [166]:
#df = qry("SELECT color, year FROM hydrants")
#df[df.color == "blue"]

,color,year
3,blue,2010
4,blue,2014
5,blue,2018


In [167]:
#qry("SELECT year FROM hydrants WHERE owner='private' AND active")

,year
0,2001


In [168]:
#df = qry("SELECT year, style, active FROM hydrants")
#df[df.active == 1]["style"]

2    Pacer
3    Pacer
4    Pacer
5    Pacer
Name: style, dtype: object

In [169]:
hydrants

,year,color,style,owner,alt,active
0,1999,red,K-81,private,1179,0
1,2000,red,M-3,public,1065,0
2,2001,green,Pacer,private,1058,1
3,2010,blue,Pacer,public,1081,1
4,2014,blue,Pacer,public,1052,1
5,2018,blue,Pacer,public,1109,1


In [170]:
#hydrants["color"].value_counts()

blue     3
red      2
green    1
Name: color, dtype: int64

In [171]:
#qry("""SELECT color, COUNT(*) FROM hydrants WHERE active GROUP BY color""")

,color,COUNT(*)
0,blue,3
1,green,1


In [172]:
#qry("""SELECT color, COUNT(*) AS count FROM hydrants GROUP BY color HAVING count > 1""")

,color,count
0,blue,3
1,red,2


In [173]:
#qry("""SELECT color, COUNT(*) AS count
    FROM hydrants WHERE year >= 2000
    GROUP BY color HAVING count < 2""")

,color,count
0,green,1
1,red,1
